# 🧠 Predictive Modeling 🧠

---

<div class="alert alert-block alert-info">
Table of Contents: <br>
    
<ul>
    <li>1. <a href="#1.-Setup">Setup</a></li>
    <li>📂 2. <a href="#%F0%9F%93%82-2.-Load-&-Preview-Data">Load & Preview Data</a></li>
    <li>🔧 3. <a href="#%F0%9F%94%A7-3.-Preprocessing">Preprocessing</a>
        <ul>
            <li>3.1. <a href="#3.1.-Splitting-the-Data">Splitting the Data</a></li>
            <li>3.2. <a href="#3.2.-Pipeline">Pipeline</a></li>
        </ul>
    </li>
    <li>😶‍🌫️ 4. <a href="#%F0%9F%98%B6%E2%80%8D%F0%9F%8C%AB%EF%B8%8F-4.-Baseline">Baseline</a></li>
    <li>🤔 5. <a href="#%F0%9F%A4%94-5.-Model-Selection">Model Selection</a></li>
    <li>🚧 6. <a href="#%F0%9F%9A%A7-6.-Feature-Engineering">Feature Engineering</a></li>
    <li>🧪 7. <a href="#%F0%9F%A7%AA-7.-Hyperparameter-Sweeping">Hyperparameter Sweeping</a></li>
    <li>🔬 8. <a href="#%F0%9F%94%AC-8.-Ensembling">Ensembling</a></li>
    <li>9. <a href="#9.-Conclusion">Conclusion</a></li>
</ul>
</div>

<h3><center>🛑 <b>Disclaimer</b> 🛑</center></h3>

From my understanding, predictive modeling is more than just taking whatever data is available and passing it through a random model. I'm still learning lots about this! Here's what I know:

* it all goes back to the data and the quality of data; models are limited by their data
* there is no free lunch; no one model works perfectly
* generally larger datasets improve performance (at least in DL)
* rather than optimize for performance, business context and use case and other real-world trade-offs have to be considered
    * is this model interpretable?
    * how fast is it?
    * how well does it integrate into the existing infrastructure?
    * does this model achieve comparable performance?
    * is it ethical? where was the data sourced? is it ok to use this data?
    * are there unintended effects?
* are there ways to solve/tackle this problem *without* machine learning?
* any hardware or financial or other limitations?

The list goes on, but generally these considerations fall into data, model, context, or deployment problems (from what I understand).

For this particular dataset, we face the following problems:
* insufficient sample size
* not enough predictive features
* a few imbalanced classes (full-time vs internships, etc)
* data based largely in CA
* some roles aren't data scientist roles
* revenue has large amount of unknown
* (my guess) the current features are not predictive enough for salary estimate
* categorical columns have very rare unique values

**Note**: Though I don't believe this dataset to be suitable for modeling, I will still treat this like a model-able task.

## 1. Setup

In [115]:
# General imports.
import os

# Specific imports.
import numpy as np
import pandas as pd

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LinearRegression

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

## 📂 2. Load & Preview Data

Let's load in our data and check if there are any immediate problems.

In [2]:
df = pd.read_csv("../input/joblisting_cleaned.csv")

In [3]:
df

,company,rating,headquarters,salary estimate,job type,size,type,sector,revenue
0,Indeed,4.3,"San Francisco, CA",209.0,Full-time,10000+ employees,company - private,information technology,$2 to $5 billion (usd)
1,Indeed,4.3,"San Francisco, CA",143.0,Full-time,10000+ employees,company - private,information technology,$2 to $5 billion (usd)
2,Abl Schools,4.1,"San Francisco, CA",140.0,Full-time,1 to 50 employees,company - private,business services,unknown / non-applicable
3,Amazon,3.8,"Palo Alto, CA",115.0,Full-time,10000+ employees,company - public,information technology,$10+ billion (usd)
4,Thermo Fisher - America,3.8,"San Francisco, CA",134.5,Full-time,10000+ employees,company - public,biotech & pharmaceuticals,$10+ billion (usd)
...,...,...,...,...,...,...,...,...,...
1696,Kaiser Permanente,4.0,"Oakland, CA",101.5,Full-time,10000+ employees,nonprofit organization,health care,$10+ billion (usd)
1697,California State University,4.3,"San Francisco, CA",81.0,Full-time,201 to 500 employees,college / university,education,$1 to $5 million (usd)
1698,Adobe,4.4,"San Francisco, CA",151.5,Full-time,10000+ employees,company - public,information technology,$5 to $10 billion (usd)
1699,First Republic Bank,4.3,"San Francisco, CA",112.5,Full-time,1001 to 5000 employees,company - public,finance,$10+ billion (usd)


In [4]:
df.shape

(1701, 9)

In [5]:
df.columns

Index(['company', 'rating', 'headquarters', 'salary estimate', 'job type',
       'size', 'type', 'sector', 'revenue'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1701 entries, 0 to 1700
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   company          1701 non-null   object 
 1   rating           1701 non-null   float64
 2   headquarters     1701 non-null   object 
 3   salary estimate  1701 non-null   float64
 4   job type         1701 non-null   object 
 5   size             1701 non-null   object 
 6   type             1701 non-null   object 
 7   sector           1701 non-null   object 
 8   revenue          1701 non-null   object 
dtypes: float64(2), object(7)
memory usage: 119.7+ KB


In [7]:
df.rating.unique()

array([4.3, 4.1, 3.8, 3.4, 4.8, 4.6, 4. , 3.2, 4.5, 4.2, 4.7, 4.4, 4.9,
       5. , 3.7, 3.5, 3.6, 3.9, 2.6, 2.8, 3.3, 3.1, 3. , 2.9, 2.5, 2.3,
       2.7, 2.1])

In [8]:
df.headquarters.unique()

array(['San Francisco, CA', 'Palo Alto, CA', 'Menlo Park, CA',
       'Sunnyvale, CA', 'Mountain View, CA', 'Newark, CA', 'Oakland, CA',
       'Redwood City, CA', 'Cupertino, CA', 'Burlingame, CA',
       'Emeryville, CA', 'South San Francisco, CA', 'San Jose, CA',
       'San Mateo, CA', 'Santa Clara, CA', 'Los Gatos, CA',
       'Sausalito, CA', 'Brisbane, CA', 'Elk Grove, CA',
       'Foster City, CA', 'Pittsburg, CA', 'Sacramento, CA',
       'Fremont, CA', 'Concord, CA', 'East Palo Alto, CA',
       'Patterson, CA', 'San Bruno, CA', 'Pleasanton, CA', 'Hercules, CA',
       'Berkeley, CA', 'Morgan Hill, CA', 'San Ramon, CA', 'Milpitas, CA',
       'San Carlos, CA', 'Alameda, CA', 'Scotts Valley, CA',
       'Livermore, CA', 'Petaluma, CA', 'Greenbrae, CA',
       'Rancho Cordova, CA', 'Saint Helena, CA', 'Oakdale, CA',
       'Clearlake, CA', 'Walnut Creek, CA', 'Lodi, CA', 'Dublin, CA',
       'Los Altos, CA', 'Stanford, CA', 'Richmond, CA', 'Hayward, CA',
       'Martinez, CA', 

In [9]:
df["salary estimate"].min(), df["salary estimate"].max()

(24.5, 209.0)

In [10]:
df["job type"].unique()

array(['Full-time', 'Part-time', 'Contract', 'Internship'], dtype=object)

In [11]:
df["size"].unique()

array(['10000+ employees', '1 to 50 employees', '201 to 500 employees',
       '51 to 200 employees', '1001 to 5000 employees',
       '501 to 1000 employees', '5001 to 10000 employees'], dtype=object)

In [12]:
df["type"].unique()

array(['company - private', 'company - public', 'nonprofit organization',
       'subsidiary or business segment', 'government',
       'college / university', 'self-employed', 'contract', 'hospital',
       'school / school district'], dtype=object)

In [13]:
df["sector"].unique()

array(['information technology', 'business services',
       'biotech & pharmaceuticals', 'finance', 'retail', 'insurance',
       'manufacturing', 'non-profit', 'health care',
       'oil, gas, energy & utilities', 'travel & tourism', 'government',
       'media', 'accounting & legal', 'arts, entertainment & recreation',
       'real estate', 'telecommunications', 'transportation & logistics',
       'education', 'construction, repair & maintenance',
       'agriculture & forestry', 'restaurants, bars & food services'],
      dtype=object)

In [14]:
df["revenue"].unique()

array(['$2 to $5 billion (usd)', 'unknown / non-applicable',
       '$10+ billion (usd)', '$25 to $50 million (usd)',
       '$10 to $25 million (usd)', 'less than $1 million (usd)',
       '$100 to $500 million (usd)', '$5 to $10 million (usd)',
       '$500 million to $1 billion (usd)', '$1 to $5 million (usd)',
       '$50 to $100 million (usd)', '$1 to $2 billion (usd)',
       '$5 to $10 billion (usd)'], dtype=object)

Alright, from an immediate inspection, it looks like we don't have any NaNs. This is good! Our cleaning pipeline works. The only nuance I can see is the "unknown / non-applicable" category for the "revenue" column.

## 🔧 3. Preprocessing

Let's set up a preprocessing pipeline. This data puts us in a precarious position. We preprocess as usual except the numerous categorical features have very rare unique values. How do we combat this?
* we can `fit` the pipeline to the entire dataset before actually transforming any data (whether that data be test, val, or train)
    * however, this has the problem of leveraging data your training pipeline *shouldn't* be seeing
* for unique categorical values that occur less than a specified threshold, we can convert it to "other"
    * basically, we fit the pipeline to training data, any rare occurrences in the train data below some threshold is converted to an "other" class
    * though do note that during test time, we would just be grouping any unfamiliar unique categorical value for a feature into "other"
    
Let's go with the 2nd method. It is more generalizable to test data. Though, of course, by grouping rare occurrences together, we are losing out on a good chunk of data. :(

### 3.1. Splitting the Data

In [15]:
y = df["salary estimate"]
X = df.drop(columns=["salary estimate"])

In [16]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

In [18]:
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(1377, 8) (153, 8) (171, 8) (1377,) (153,) (171,)


### 3.2. Pipeline

In [46]:
df.columns

Index(['company', 'rating', 'headquarters', 'salary estimate', 'job type',
       'size', 'type', 'sector', 'revenue'],
      dtype='object')

In [53]:
remove_attribs = ["company"]
num_attribs = ["rating"]
cat_attribs = ["headquarters", "job type", "size", "type", "sector", "revenue"]

In [54]:
num_pipeline = Pipeline([
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("one_hot", OneHotEncoder())
])

In [85]:
class CustomRemover(BaseEstimator, TransformerMixin):
    
    def __init__(self, useless_attribs):
        self.useless_attribs = useless_attribs
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy = X_copy.drop(self.useless_attribs, axis=1)
        return X_copy
    
class CatGrouper(BaseEstimator, TransformerMixin):
    
    def __init__(self, cols, threshold=0.1):
        self.cols = cols
        self.threshold = threshold
        self.cat_groups = {}
        
    def fit(self, X, y=None):
        for attrib in self.cols:
            vc = df[attrib].value_counts(normalize=True)
            thres = vc < self.threshold
            keep = vc[np.logical_not(thres)].index
            self.cat_groups[attrib] = list(keep)
        return self
    
    def _map_func(self, v, attrib):
        if v not in self.cat_groups[attrib]:
            v = "Other"
        return v
    
    def transform(self, X):
        X_copy = X.copy()
        for attrib in self.cols:
            X_copy[attrib] = X_copy[attrib].apply(self._map_func, attrib=attrib)
        return X_copy

In [86]:
base_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs)
])

preprocessing_pipeline = Pipeline([
    ("remover", CustomRemover(remove_attribs)),
    ("cat_grouper", CatGrouper(cat_attribs, threshold=0.05)),
    ("base", base_pipeline)
])

In [91]:
X_train_prep = preprocessing_pipeline.fit_transform(X_train)
X_val_prep = preprocessing_pipeline.transform(X_val)
X_test_prep = preprocessing_pipeline.transform(X_test)

Alright, we just finished with the preprocessing! It was fairly straightforward except I had to include a custom transformer for the rare categorical values. Let's start on our baseline.

## 😶‍🌫️ 4. Baseline

Before we dive straight into more complex, black-box models and before we develop more complex systems, let's try a simple baseline. This method is helpful for gauging how usable a dataset is and how complex of a model you need to tackle this dataset. Since we are aiming to predict salary estimate, this is a regression task. Let's start with a simple Linear Regression model. This model is fast but simple. It will probably be beaten by stronger methods.

And also, note, before or after preprocessing, dimensionality reduction should be considered especially if a lot of the data is not relevant. However, considering how small our dataset is, we won't DR the data.

In [93]:
X_train_prep, y_train

(<1377x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 9639 stored elements in Compressed Sparse Row format>,
 66      165.0
 1247    160.0
 892     144.0
 313     155.0
 689     142.0
         ...  
 653     139.0
 1462    151.0
 223     124.5
 1674     88.5
 1115    105.5
 Name: salary estimate, Length: 1377, dtype: float64)

In [106]:
reg = LinearRegression()
reg.fit(X_train_prep, y_train)

LinearRegression()

That was quick! We have a small dataset and this model is relatively simple. I just showed how to *train* a single model. Let's train an ensemble of linear regression models and cross-validate them to see how linear regression *really* performs on this dataset.

There are a variety of metrics to track. The 2 most popular regression metrics are **mean squared error (MSE)** and **mean absolute error (MAE)**. Choosing a metric to evaluate your model on is crucial. Here we can afford to test multiple metrics to get a sense of which one is most useful, but in certain cases, switching between metrics is costly. 

In [118]:
n_folds = 5
n_jobs = -1
verbose = 2

In [119]:
scores = cross_val_score(reg, X_val_prep, y_val, 
                         cv=n_folds, scoring="neg_mean_absolute_error", 
                         n_jobs=n_jobs, verbose=verbose)

for fold, score in zip(range(n_folds), scores):
    print(f"Fold-{fold}: {score}")
    
print(f"\nMean Score: {np.mean(scores)}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s


Fold-0: -20.093198584264933
Fold-1: -26.169084126622074
Fold-2: -21.43110252124096
Fold-3: -18.473662959962766
Fold-4: -20.3323498451929

Mean Score: -21.299879607456727


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished


In [120]:
scores = cross_val_score(reg, X_val_prep, y_val, 
                         cv=n_folds, scoring="neg_mean_squared_error", 
                         n_jobs=n_jobs, verbose=verbose)

for fold, score in zip(range(n_folds), scores):
    print(f"Fold-{fold}: {score}")
    
print(f"\nMean Score: {np.mean(scores)}")

Fold-0: -645.5809525966392
Fold-1: -935.6621520610457
Fold-2: -687.1457157617054
Fold-3: -498.10166476597925
Fold-4: -661.864560345873

Mean Score: -685.6710091062486


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


Look at the size of those numbers for MSE! They are insanely large. I think it's safe to say MAE would be better in this context for judging model performance. This is because, from our EDA, we noticed that there were outliers in the salary estimates. From what I recall, one joblisting had an estimated salary of 24.5k compared to the dataset mean of ~135k.

Let's test a few other regressors in the next section.

## 🤔 5. Model Selection

This section will describe a few more advanced regressors. We will also test them before doing a hyperparameter sweep. We want to narrow down to get the best models before sweeping and fine-tuning them.

The models we will consider for this task are:
* Decision Tree
* Random Forest
* Extra Trees
* SVM
* XGBoost
* Catboost
* LightGBM
* dense shallow neural network

Model selection is a tricky business. I didn't include common techniques like ElasticNet or Ridge or LASSO because they are similar to linear regression in that they add regularization constraint components. Here we consider speed versus accuracy. Decision Tree and SVM (because of our small dataset) are both fast. The models are a bit slower. However, the slower models make up for it by generally performing better. What's nice is that some of these implementations (i.e. LightGBM) are optimized for efficiency and speed. I predict that either XGBoost or LightGBM wins in this model selection competition!

## 🚧 6. Feature Engineering

## 🧪 7. Hyperparameter Sweeping

## 🔬 8. Ensembling

## 9. Conclusion